<a href="https://colab.research.google.com/github/Marieta3/sentiment-analysis-russian/blob/master/sentiment_analiza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analiza sentimenta tvitova na ruskom jeziku


##Importovanje potrebnih biblioteka

In [1]:
import tensorflow as tf
import tflearn
import pandas as pd
from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import TweetTokenizer
import re
from collections import Counter
from datetime import datetime
import numpy as np
from tflearn.data_utils import to_categorical

W0702 19:30:05.916542 140089435973504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0702 19:30:05.918597 140089435973504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0702 19:30:05.939906 140089435973504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0702 19:30:05.948979 140089435973504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0702 19:30:05.964372 140089435973504 deprecation_wrapper.py:119] From /usr/local/lib/

##Konstante

In [0]:
VOCAB_SIZE = 5000
MANJI_DEO = False
TEST_SIZE = 0.3

##Učitavanje podataka

In [0]:
col = 3 #kolona koja sadrzi tvit, ostale kolone nisu potrebne
rows=100000
if(MANJI_DEO):
  positive = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/positive.csv', header=None, delimiter=';')[:rows][col];
  negative = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/negative.csv', header=None, delimiter=';')[:rows][col];
else:
  positive = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/positive.csv', header=None, delimiter=';')[[col]];
  negative = pd.read_csv(
      'https://raw.githubusercontent.com/Marieta3/sentiment-analysis-russian/master/negative.csv', header=None, delimiter=';')[[col]];  

##Testiranje učitanih podataka


In [4]:
print('Pozitivnih tvitova: ' +str(len(positive)))
print('Negativnih tvitova: ' +str(len(negative)))

positive = positive.head(30000)
negative = negative.head(30000)

print('Pozitivnih tvitova: ' +str(len(positive)))
print('Negativnih tvitova: ' +str(len(negative)))

Pozitivnih tvitova: 114911
Negativnih tvitova: 111923
Pozitivnih tvitova: 30000
Negativnih tvitova: 30000


##Izvlačenje osnovnog oblika iz tokena

In [5]:
stemmer = RussianStemmer()
stem_cache={}  #smanjuje vreme izvršavanja, ako se reč već pojavljivala, samo je uzmemo iz liste
def get_stem(token):
  stem =stem_cache.get(token, None)
  if stem:
    return stem
  
  token=re.sub('[^а-яА-Я ]', '', token)  #samo slova
  stem=stemmer.stem(token)
  stem_cache[token]=stem
  return stem
get_stem('!')

''

##Tokenizacija

In [0]:
recnik = Counter()
tokenizer = TweetTokenizer()


def popuni_recnik(tweets):
  for t in tweets.values.tolist():
    #print(t[0])
    tweet=t[0]
    
    tokens=tokenizer.tokenize(tweet)
    for token in tokens:
      stem=get_stem(token)
      if stem!='':
        recnik[stem]+=1
    
start_time=datetime.now()  
popuni_recnik(positive)
popuni_recnik(negative)
end_time=datetime.now()

In [7]:
print('Потребно време за попуњавање речника: ', end_time-start_time)
print("Укупан број различитих речи: ", len(recnik))


Потребно време за попуњавање речника:  0:00:28.801885
Укупан број различитих речи:  41822


In [0]:
vocab = sorted(recnik, key=recnik.get, reverse=True)[:VOCAB_SIZE]  #сортирано по броју понављања опадајуће, првих 5000

In [9]:
print(vocab[:10])
print(vocab[1000:1020])
print(vocab[-10:])
print("Дужина скраћеног речника: ",len(vocab))

['не', 'я', 'и', 'в', 'на', 'а', 'что', 'с', 'так', 'эт']
['выуч', 'результат', 'гдет', 'збс', 'минус', 'нафиг', 'порт', 'выбра', 'вперв', 'бред', 'шутк', 'киев', 'дед', 'ктонибуд', 'выгляд', 'просмотр', 'хм', 'даш', 'возьм', 'снима']
['позад', 'джо', 'попк', 'пользу', 'лап', 'произойдет', 'ледов', 'француз', 'бяк', 'проц']
Дужина скраћеног речника:  5000


In [10]:
idx=5000-3
print("Reč: ", vocab[idx], " se pojavila ", recnik.get(vocab[idx]), " puta")




Reč:  француз  se pojavila  9  puta


In [11]:
token_to_idx = {vocab[i] : i for i in range(VOCAB_SIZE)}
print(token_to_idx.get('', None))
print(type(token_to_idx))

None
<class 'dict'>


##Pretvaranje tvita u vektor

*   vektor je duzine broja reci u recniku (5000)
*   ukoliko tvit sadrzi rec koja se nalazi u recniku, na odgovarajucem mestu u vektoru se nalazi jedinica; u suprotnom nula


In [0]:
def tweet_to_vector(tweet, print_unknown=False):
  vector=np.zeros(VOCAB_SIZE, dtype=int)
  for token in tokenizer.tokenize(tweet):
    stem=get_stem(token)
    idx=token_to_idx.get(stem, None)
    if idx is not None:
      vector[idx]=1
    else:
      if(print_unknown):
        print('Nepoznata rec: ', token)
  return vector
    

In [13]:
tw_idx=1
pos_tws=positive.values.tolist()
neg_tws=negative.values.tolist()
print("tweet: {}".format(pos_tws[tw_idx][0]))
print("tweet: {}".format(tweet_to_vector(pos_tws[tw_idx][0])[:10]))
print("tweet: {}".format(neg_tws[tw_idx][0]))
print("tweet: {}".format(tweet_to_vector(neg_tws[tw_idx][0])[:10]))

tweet: Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D
tweet: [0 0 0 0 1 0 0 0 0 0]
tweet: Коллеги сидят рубятся в Urban terror, а я из-за долбанной винды не могу :(
tweet: [1 1 0 1 0 1 0 0 0 0]


In [14]:
try:
  del train_vectors
  del test_vectors
except:
  print('Nije definisano')
  
train_size = round((len(negative) + len(positive))*(1-TEST_SIZE))
test_size = len(negative) + len(positive) - train_size

train_vectors = np.zeros((train_size, VOCAB_SIZE), dtype=np.int_)
test_vectors = np.zeros((test_size, VOCAB_SIZE), dtype=np.int_)

Nije definisano


In [0]:
tweets=[]
cnt=0
cnt2=0

train_pos_size = round(len(positive)*(1-TEST_SIZE))
train_neg_size = round(len(negative)*(1-TEST_SIZE))

for tweet in negative.values.tolist():
  #print(tweet[0])
  tweets.append(tweet[0])
  if cnt < train_neg_size:
    train_vectors[cnt]=tweet_to_vector(tweet[0])
    cnt+=1
  else:
    test_vectors[cnt2]=tweet_to_vector(tweet[0])
    cnt2+=1
for tweet in positive.values.tolist():
  tweets.append(tweet[0])
  if cnt < train_neg_size + train_pos_size:
    train_vectors[cnt]=tweet_to_vector(tweet[0])
    cnt+=1
  else:
    test_vectors[cnt2]=tweet_to_vector(tweet[0])
    cnt2+=1


In [16]:
print(len(train_vectors))
print(len(train_vectors[0]))
print(train_vectors)

42000
5000
[[0 1 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


##Brisanje vektora tvitova

In [0]:
del train_vectors
del test_vectors

##Labele

In [0]:
#labels = np.append(np.zeros(len(negative), dtype=np.int_), np.ones(len(positive), dtype=np.int_))

train_labels = np.append(
    np.zeros(train_neg_size, dtype=np.int_), 
    np.ones(train_pos_size, dtype=np.int_))

test_labels = np.append(
    np.zeros(len(negative) - train_neg_size, dtype=np.int_), 
    np.ones(len(positive) - train_pos_size, dtype=np.int_))

In [0]:
y_train = to_categorical(train_labels, 2)
y_test = to_categorical(test_labels, 2)


##Kreiranje modela

In [0]:
def build_model(learning_rate=0.1):
    tf.reset_default_graph()
    
    net = tflearn.input_data([None, VOCAB_SIZE])
    net = tflearn.fully_connected(net, 125, activation='ReLU')
    net = tflearn.fully_connected(net, 25, activation='ReLU')
    net = tflearn.fully_connected(net, 2, activation='softmax')
    regression = tflearn.regression(
        net, 
        optimizer='sgd', 
        learning_rate=learning_rate, 
        loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    return model

In [0]:

model = build_model(learning_rate=0.75)

In [21]:
model.fit(
    train_vectors, 
    y_train, 
    validation_set=0.1, 
    show_metric=True, 
    batch_size=128, 
    n_epoch=3)

Training Step: 887  | total loss: 0.54931 | time: 5.186s
| SGD | epoch: 003 | loss: 0.54931 - acc: 0.7101 -- iter: 37760/37800
Training Step: 888  | total loss: 0.54977 | time: 6.229s
| SGD | epoch: 003 | loss: 0.54977 - acc: 0.7086 | val_loss: 0.57201 - val_acc: 0.6962 -- iter: 37800/37800
--


##Testiranje

In [22]:
predictions = (np.array(model.predict(test_vectors))[:,0] >= 0.5).astype(np.int_)
accuracy = np.mean(predictions == y_test[:,0], axis=0)
print("Accuracy: ", accuracy)

Accuracy:  0.6950555555555555


In [0]:
def test_tweet(tweet):
    tweet_vector = tweet_to_vector(tweet, True)
    positive_prob = model.predict([tweet_vector])[0][1]
    print('Original tweet: {}'.format(tweet))
    print('P(positive) = {:.5f}. Result: '.format(positive_prob), 
          'Positive' if positive_prob > 0.5 else 'Negative')

In [0]:
def test_tweet_number(idx):
    test_tweet(tweets[idx])

In [36]:
test_tweet_number(40003)

Nepoznata rec:  *
Nepoznata rec:  -
Nepoznata rec:  *
Nepoznata rec:  ;)
Nepoznata rec:  куколки
Nepoznata rec:  :
Nepoznata rec:  *
Nepoznata rec:  http://t.co/1Hnl9DWxzh
Original tweet: наши ночии *-* круты были очень ;) скучаю куколки моии :* http://t.co/1Hnl9DWxzh
P(positive) = 0.74390. Result:  Positive


##Proba

In [45]:
tweets_for_testing = [
    "меня оштрафовали по дороге домой!",
    "Я люблю мороженое!",
    "Я не люблю мороженое",
    "меня обманули",
    "меня обманули позитив",
    "меня он очень бесить",
    "меня он не бесить"
]
for tweet in tweets_for_testing:
    test_tweet(tweet) 
    print("---------")

Nepoznata rec:  оштрафовали
Nepoznata rec:  !
Original tweet: меня оштрафовали по дороге домой!
P(positive) = 0.20806. Result:  Negative
---------
Nepoznata rec:  !
Original tweet: Я люблю мороженое!
P(positive) = 0.77530. Result:  Positive
---------
Original tweet: Я не люблю мороженое
P(positive) = 0.35009. Result:  Negative
---------
Nepoznata rec:  обманули
Original tweet: меня обманули
P(positive) = 0.25979. Result:  Negative
---------
Nepoznata rec:  обманули
Original tweet: меня обманули позитив
P(positive) = 0.28882. Result:  Negative
---------
Original tweet: меня он очень бесить
P(positive) = 0.16582. Result:  Negative
---------
Original tweet: меня он не бесить
P(positive) = 0.15639. Result:  Negative
---------
